# Diagnostic 16: Comprehensive Grid Search - ALL 9 Strategies

**Purpose:** Optimize parameters for ALL trading strategies with small farmer costs

**Strategies:**
1. ImmediateSaleStrategy
2. EqualBatchStrategy
3. PriceThresholdStrategy
4. MovingAverageStrategy
5. PriceThresholdPredictive
6. MovingAveragePredictive
7. ExpectedValueStrategy
8. ConsensusStrategy
9. RiskAdjustedStrategy

**Key Configuration:**
- Small farmer costs: 0.005% storage/day, 0.01% transaction
- Coarse grid for initial optimization
- All strategies imported from all_strategies_pct.py

**Expected Results:**
- Optimal parameters for each strategy
- Clear ranking with realistic costs
- Matched pairs showing prediction value-add

In [ ]:
%run ../00_setup_and_config

In [ ]:
# Install Optuna for intelligent hyperparameter optimization
%pip install optuna --quiet

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
import optuna
from optuna.samplers import TPESampler
import importlib.util

print("="*80)
print("DIAGNOSTIC 16: OPTUNA OPTIMIZATION - 99.6% FASTER")
print("="*80)
print("\n1,800 trials vs 520,000 grid combinations")

## Load Strategies

In [ ]:
# Force fresh reload
if 'all_strategies_pct' in sys.modules:
    del sys.modules['all_strategies_pct']

spec = importlib.util.spec_from_file_location("all_strategies_pct", "all_strategies_pct.py")
strategies_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(strategies_module)

ImmediateSaleStrategy = strategies_module.ImmediateSaleStrategy
EqualBatchStrategy = strategies_module.EqualBatchStrategy
PriceThresholdStrategy = strategies_module.PriceThresholdStrategy
MovingAverageStrategy = strategies_module.MovingAverageStrategy
PriceThresholdPredictive = strategies_module.PriceThresholdPredictive
MovingAveragePredictive = strategies_module.MovingAveragePredictive
ExpectedValueStrategy = strategies_module.ExpectedValueStrategy
ConsensusStrategy = strategies_module.ConsensusStrategy
RiskAdjustedStrategy = strategies_module.RiskAdjustedStrategy

print("✓ Loaded all 9 strategies from all_strategies_pct.py")

## Load Data with Small Farmer Costs

In [ ]:
COMMODITY = 'coffee'
MODEL_VERSION = 'synthetic_acc90'

DATA_PATHS = get_data_paths(COMMODITY, MODEL_VERSION)
COMMODITY_CONFIG = COMMODITY_CONFIGS[COMMODITY]

# OVERRIDE: Small farmer realistic costs
COMMODITY_CONFIG['storage_cost_pct_per_day'] = 0.005  # Was 0.025%
COMMODITY_CONFIG['transaction_cost_pct'] = 0.01       # Was 0.25%

print("Loading data...")
prices_table = get_data_paths(COMMODITY)['prices_prepared']
prices = spark.table(prices_table).toPandas()
prices['date'] = pd.to_datetime(prices['date'])

matrices_path = DATA_PATHS['prediction_matrices']
with open(matrices_path, 'rb') as f:
    prediction_matrices = pickle.load(f)
prediction_matrices = {pd.to_datetime(k): v for k, v in prediction_matrices.items()}

print(f"✓ Loaded {len(prices)} price records")
print(f"✓ Loaded {len(prediction_matrices)} prediction matrices")
print(f"\nSmall Farmer Costs:")
print(f"  Storage: {COMMODITY_CONFIG['storage_cost_pct_per_day']}% per day")
print(f"  Transaction: {COMMODITY_CONFIG['transaction_cost_pct']}%")

## Backtest Engine

In [ ]:
class DiagnosticBacktestEngine:
    """Backtest engine for grid search"""
    
    def __init__(self, prices_df, prediction_matrices, commodity_config):
        self.prices = prices_df
        self.prediction_matrices = prediction_matrices
        self.config = commodity_config
        
    def run_backtest(self, strategy, initial_inventory=50.0):
        inventory = initial_inventory
        trades = []
        total_revenue = 0
        total_transaction_costs = 0
        total_storage_costs = 0
        
        strategy.reset()
        strategy.set_harvest_start(0)
        
        for day in range(len(self.prices)):
            current_date = self.prices.iloc[day]['date']
            current_price = self.prices.iloc[day]['price']
            price_history = self.prices.iloc[:day+1].copy()
            predictions = self.prediction_matrices.get(current_date, None)
            
            decision = strategy.decide(
                day=day,
                inventory=inventory,
                current_price=current_price,
                price_history=price_history,
                predictions=predictions
            )
            
            if decision['action'] == 'SELL' and decision['amount'] > 0:
                amount = min(decision['amount'], inventory)
                price_per_ton = current_price * 20
                revenue = amount * price_per_ton
                transaction_cost = revenue * (self.config['transaction_cost_pct'] / 100)
                
                total_revenue += revenue
                total_transaction_costs += transaction_cost
                inventory -= amount
                
                trades.append({
                    'day': day,
                    'amount': amount,
                    'price': current_price,
                    'revenue': revenue
                })
            
            if inventory > 0:
                avg_price = self.prices.iloc[:day+1]['price'].mean()
                price_per_ton = avg_price * 20
                storage_cost = inventory * price_per_ton * (self.config['storage_cost_pct_per_day'] / 100)
                total_storage_costs += storage_cost
        
        net_earnings = total_revenue - total_transaction_costs - total_storage_costs
        
        return {
            'net_earnings': net_earnings,
            'total_revenue': total_revenue,
            'num_trades': len(trades),
            'storage_costs': total_storage_costs,
            'final_inventory': inventory
        }

engine = DiagnosticBacktestEngine(prices, prediction_matrices, COMMODITY_CONFIG)
print("✓ Backtest engine ready")

## Optuna Search Spaces - ALL 9 Strategies

In [ ]:
def get_search_space(trial, strategy_name):
    '''COMPLETE parameter coverage for ALL strategies'''
    if strategy_name == 'immediate_sale':
        return {'min_batch_size': trial.suggest_float('min_batch_size', 3.0, 10.0), 'sale_frequency_days': trial.suggest_int('sale_frequency_days', 5, 14)}
    
    elif strategy_name == 'equal_batch':
        return {'batch_size': trial.suggest_float('batch_size', 0.15, 0.30), 'frequency_days': trial.suggest_int('frequency_days', 20, 35)}
    
    elif strategy_name == 'price_threshold':
        return {'threshold_pct': trial.suggest_float('threshold_pct', 0.02, 0.07), 'batch_baseline': trial.suggest_float('batch_baseline', 0.20, 0.35), 'batch_overbought_strong': trial.suggest_float('batch_overbought_strong', 0.30, 0.40), 'batch_overbought': trial.suggest_float('batch_overbought', 0.25, 0.35), 'batch_strong_trend': trial.suggest_float('batch_strong_trend', 0.15, 0.25), 'rsi_overbought': trial.suggest_int('rsi_overbought', 65, 75), 'rsi_moderate': trial.suggest_int('rsi_moderate', 60, 70), 'adx_strong': trial.suggest_int('adx_strong', 20, 30), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 10), 'max_days_without_sale': trial.suggest_int('max_days_without_sale', 45, 75)}
    
    elif strategy_name == 'moving_average':
        return {'ma_period': trial.suggest_int('ma_period', 20, 35), 'batch_baseline': trial.suggest_float('batch_baseline', 0.20, 0.30), 'batch_strong_momentum': trial.suggest_float('batch_strong_momentum', 0.15, 0.25), 'batch_overbought': trial.suggest_float('batch_overbought', 0.25, 0.35), 'batch_overbought_strong': trial.suggest_float('batch_overbought_strong', 0.30, 0.40), 'rsi_overbought': trial.suggest_int('rsi_overbought', 65, 75), 'rsi_min': trial.suggest_int('rsi_min', 40, 50), 'adx_strong': trial.suggest_int('adx_strong', 20, 30), 'adx_weak': trial.suggest_int('adx_weak', 15, 25), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 10), 'max_days_without_sale': trial.suggest_int('max_days_without_sale', 45, 75)}
    
    elif strategy_name == 'price_threshold_predictive':
        return {'threshold_pct': trial.suggest_float('threshold_pct', 0.02, 0.07), 'batch_baseline': trial.suggest_float('batch_baseline', 0.20, 0.35), 'batch_overbought_strong': trial.suggest_float('batch_overbought_strong', 0.30, 0.40), 'batch_overbought': trial.suggest_float('batch_overbought', 0.25, 0.35), 'batch_strong_trend': trial.suggest_float('batch_strong_trend', 0.15, 0.25), 'rsi_overbought': trial.suggest_int('rsi_overbought', 65, 75), 'rsi_moderate': trial.suggest_int('rsi_moderate', 60, 70), 'adx_strong': trial.suggest_int('adx_strong', 20, 30), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 10), 'max_days_without_sale': trial.suggest_int('max_days_without_sale', 45, 75), 'min_net_benefit_pct': trial.suggest_float('min_net_benefit_pct', 0.3, 1.0), 'high_confidence_cv': trial.suggest_float('high_confidence_cv', 0.03, 0.08), 'scenario_shift_aggressive': trial.suggest_int('scenario_shift_aggressive', 1, 2), 'scenario_shift_conservative': trial.suggest_int('scenario_shift_conservative', 1, 2)}
    
    elif strategy_name == 'moving_average_predictive':
        return {'ma_period': trial.suggest_int('ma_period', 20, 35), 'batch_baseline': trial.suggest_float('batch_baseline', 0.20, 0.30), 'batch_strong_momentum': trial.suggest_float('batch_strong_momentum', 0.15, 0.25), 'batch_overbought': trial.suggest_float('batch_overbought', 0.25, 0.35), 'batch_overbought_strong': trial.suggest_float('batch_overbought_strong', 0.30, 0.40), 'rsi_overbought': trial.suggest_int('rsi_overbought', 65, 75), 'rsi_min': trial.suggest_int('rsi_min', 40, 50), 'adx_strong': trial.suggest_int('adx_strong', 20, 30), 'adx_weak': trial.suggest_int('adx_weak', 15, 25), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 10), 'max_days_without_sale': trial.suggest_int('max_days_without_sale', 45, 75), 'min_net_benefit_pct': trial.suggest_float('min_net_benefit_pct', 0.3, 1.0), 'high_confidence_cv': trial.suggest_float('high_confidence_cv', 0.03, 0.08), 'scenario_shift_aggressive': trial.suggest_int('scenario_shift_aggressive', 1, 2), 'scenario_shift_conservative': trial.suggest_int('scenario_shift_conservative', 1, 2)}
    
    elif strategy_name == 'expected_value':
        return {'min_net_benefit_pct': trial.suggest_float('min_net_benefit_pct', 0.3, 1.0), 'negative_threshold_pct': trial.suggest_float('negative_threshold_pct', -0.5, -0.1), 'high_confidence_cv': trial.suggest_float('high_confidence_cv', 0.03, 0.08), 'medium_confidence_cv': trial.suggest_float('medium_confidence_cv', 0.10, 0.15), 'strong_trend_adx': trial.suggest_int('strong_trend_adx', 20, 25), 'batch_positive_confident': trial.suggest_float('batch_positive_confident', 0.0, 0.05), 'batch_positive_uncertain': trial.suggest_float('batch_positive_uncertain', 0.10, 0.20), 'batch_marginal': trial.suggest_float('batch_marginal', 0.15, 0.20), 'batch_negative_mild': trial.suggest_float('batch_negative_mild', 0.25, 0.30), 'batch_negative_strong': trial.suggest_float('batch_negative_strong', 0.35, 0.40), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 7), 'baseline_batch': trial.suggest_float('baseline_batch', 0.15, 0.20), 'baseline_frequency': trial.suggest_int('baseline_frequency', 25, 30)}
    
    elif strategy_name == 'consensus':
        return {'consensus_threshold': trial.suggest_float('consensus_threshold', 0.60, 0.75), 'very_strong_consensus': trial.suggest_float('very_strong_consensus', 0.80, 0.85), 'moderate_consensus': trial.suggest_float('moderate_consensus', 0.55, 0.60), 'min_return': trial.suggest_float('min_return', 0.02, 0.05), 'min_net_benefit_pct': trial.suggest_float('min_net_benefit_pct', 0.3, 0.7), 'high_confidence_cv': trial.suggest_float('high_confidence_cv', 0.03, 0.08), 'batch_strong_consensus': trial.suggest_float('batch_strong_consensus', 0.0, 0.05), 'batch_moderate': trial.suggest_float('batch_moderate', 0.10, 0.20), 'batch_weak': trial.suggest_float('batch_weak', 0.25, 0.30), 'batch_bearish': trial.suggest_float('batch_bearish', 0.35, 0.40), 'evaluation_day': trial.suggest_int('evaluation_day', 10, 14), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 7)}
    
    elif strategy_name == 'risk_adjusted':
        return {'min_return': trial.suggest_float('min_return', 0.02, 0.05), 'min_net_benefit_pct': trial.suggest_float('min_net_benefit_pct', 0.3, 0.7), 'max_uncertainty_low': trial.suggest_float('max_uncertainty_low', 0.03, 0.08), 'max_uncertainty_medium': trial.suggest_float('max_uncertainty_medium', 0.10, 0.20), 'max_uncertainty_high': trial.suggest_float('max_uncertainty_high', 0.25, 0.35), 'strong_trend_adx': trial.suggest_int('strong_trend_adx', 20, 25), 'batch_low_risk': trial.suggest_float('batch_low_risk', 0.0, 0.05), 'batch_medium_risk': trial.suggest_float('batch_medium_risk', 0.10, 0.15), 'batch_high_risk': trial.suggest_float('batch_high_risk', 0.25, 0.30), 'batch_very_high_risk': trial.suggest_float('batch_very_high_risk', 0.35, 0.40), 'evaluation_day': trial.suggest_int('evaluation_day', 10, 14), 'cooldown_days': trial.suggest_int('cooldown_days', 5, 7)}
    
    else:
        raise ValueError(f'Unknown: {strategy_name}')

print('✓ COMPLETE search spaces - all parameters for all 9 strategies')

def optimize_strategy(strategy_class, strategy_name, engine, n_trials=200, n_jobs=1, needs_costs=False):
    """
    Optimize strategy using Optuna.
    
    Note: Running sequentially (n_jobs=1) to avoid any SQLite/parallelization issues.
    Set n_jobs higher if your environment supports it.
    """
    print(f'\n{"="*80}\nOptimizing {strategy_name}: {n_trials} trials\n{"="*80}')
    
    # Create study with NO storage backend (pure in-memory)
    study = optuna.create_study(
        direction='maximize', 
        sampler=TPESampler(seed=42)
    )
    
    def objective(trial):
        params = get_search_space(trial, strategy_name)
        if needs_costs:
            params['storage_cost_pct_per_day'] = COMMODITY_CONFIG['storage_cost_pct_per_day']
            params['transaction_cost_pct'] = COMMODITY_CONFIG['transaction_cost_pct']
        try:
            strategy = strategy_class(**params)
            result = engine.run_backtest(strategy)
            return result['net_earnings']
        except Exception as e:
            print(f"  Trial {trial.number} failed: {e}")
            return -1e9
    
    # Run optimization (n_jobs=1 for sequential, increase if environment supports it)
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, show_progress_bar=True)
    
    print(f'\n✓ Best: ${study.best_value:,.2f}')
    print(f'  Completed {len(study.trials)} trials')
    
    return study.best_params, study

print('✓ Optimization ready (in-memory, no SQLite)')

In [ ]:
def optimize_strategy(strategy_class, strategy_name, engine, n_trials=200, n_workers=8, needs_costs=False):
    print(f'\n{"="*80}\nOptimizing {strategy_name}: {n_trials} trials, {n_workers} workers\n{"="*80}')
    storage_dir = f'/dbfs/tmp/optuna_{strategy_name}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    os.makedirs(storage_dir, exist_ok=True)
    storage_url = f'sqlite:///{storage_dir}/study.db'
    study = optuna.create_study(study_name=strategy_name, storage=storage_url, direction='maximize', sampler=TPESampler(seed=42), load_if_exists=True)
    
    def objective(trial):
        params = get_search_space(trial, strategy_name)
        if needs_costs:
            params['storage_cost_pct_per_day'] = COMMODITY_CONFIG['storage_cost_pct_per_day']
            params['transaction_cost_pct'] = COMMODITY_CONFIG['transaction_cost_pct']
        try:
            strategy = strategy_class(**params)
            result = engine.run_backtest(strategy)
            return result['net_earnings']
        except Exception as e:
            return -1e9
    
    def run_worker(worker_id):
        worker_study = optuna.load_study(study_name=strategy_name, storage=storage_url, sampler=TPESampler(seed=42 + worker_id))
        worker_study.optimize(objective, n_trials=n_trials//n_workers, show_progress_bar=False)
        return worker_id
    
    rdd = spark.sparkContext.parallelize(range(n_workers), n_workers)
    rdd.map(run_worker).collect()
    final_study = optuna.load_study(study_name=strategy_name, storage=storage_url)
    
    try:
        import shutil
        shutil.rmtree(storage_dir)
    except:
        pass
    
    print(f'Best: ${final_study.best_value:,.2f}')
    return final_study.best_params, final_study

print('✓ Optimization ready')

## Run Optimizations - ALL 9 Strategies

In [ ]:
all_results = {}

params, study = optimize_strategy(ImmediateSaleStrategy, 'immediate_sale', engine, n_trials=200, n_jobs=8, needs_costs=False)
all_results['immediate_sale'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(ImmediateSaleStrategy, 'immediate_sale', engine, n_trials=200, n_workers=8, needs_costs=False)
all_results['immediate_sale'] = (params, study.best_value)

params, study = optimize_strategy(EqualBatchStrategy, 'equal_batch', engine, n_trials=200, n_jobs=8, needs_costs=False)
all_results['equal_batch'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(EqualBatchStrategy, 'equal_batch', engine, n_trials=200, n_workers=8, needs_costs=False)
all_results['equal_batch'] = (params, study.best_value)

params, study = optimize_strategy(PriceThresholdStrategy, 'price_threshold', engine, n_trials=200, n_jobs=8, needs_costs=False)
all_results['price_threshold'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(PriceThresholdStrategy, 'price_threshold', engine, n_trials=200, n_workers=8, needs_costs=False)
all_results['price_threshold'] = (params, study.best_value)

params, study = optimize_strategy(MovingAverageStrategy, 'moving_average', engine, n_trials=200, n_jobs=8, needs_costs=False)
all_results['moving_average'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(MovingAverageStrategy, 'moving_average', engine, n_trials=200, n_workers=8, needs_costs=False)
all_results['moving_average'] = (params, study.best_value)

params, study = optimize_strategy(PriceThresholdPredictive, 'price_threshold_predictive', engine, n_trials=200, n_jobs=8, needs_costs=True)
all_results['price_threshold_predictive'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(PriceThresholdPredictive, 'price_threshold_predictive', engine, n_trials=200, n_workers=8, needs_costs=True)
all_results['price_threshold_predictive'] = (params, study.best_value)

params, study = optimize_strategy(MovingAveragePredictive, 'moving_average_predictive', engine, n_trials=200, n_jobs=8, needs_costs=True)
all_results['moving_average_predictive'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(MovingAveragePredictive, 'moving_average_predictive', engine, n_trials=200, n_workers=8, needs_costs=True)
all_results['moving_average_predictive'] = (params, study.best_value)

params, study = optimize_strategy(ExpectedValueStrategy, 'expected_value', engine, n_trials=200, n_jobs=8, needs_costs=True)
all_results['expected_value'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(ExpectedValueStrategy, 'expected_value', engine, n_trials=200, n_workers=8, needs_costs=True)
all_results['expected_value'] = (params, study.best_value)

params, study = optimize_strategy(ConsensusStrategy, 'consensus', engine, n_trials=200, n_jobs=8, needs_costs=True)
all_results['consensus'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(ConsensusStrategy, 'consensus', engine, n_trials=200, n_workers=8, needs_costs=True)
all_results['consensus'] = (params, study.best_value)

params, study = optimize_strategy(RiskAdjustedStrategy, 'risk_adjusted', engine, n_trials=200, n_jobs=8, needs_costs=True)
all_results['risk_adjusted'] = (params, study.best_value)

In [ ]:
params, study = optimize_strategy(RiskAdjustedStrategy, 'risk_adjusted', engine, n_trials=200, n_workers=8, needs_costs=True)
all_results['risk_adjusted'] = (params, study.best_value)

## Summary Analysis

In [ ]:
print("\n" + "="*80)
print("OPTUNA OPTIMIZATION COMPLETE - ALL 9 STRATEGIES")
print("="*80)

# Create summary
summary = []
for name, (params, best_value) in all_results.items():
    summary.append({
        'strategy': name,
        'net_earnings': best_value,
        'params': params
    })

# Sort by earnings
summary.sort(key=lambda x: x['net_earnings'], reverse=True)

# Display ranking
print("\nRanking by Net Earnings:")
print(f"{'Rank':<6} {'Strategy':<35} {'Net Earnings':>15}")
print("-" * 80)

for i, s in enumerate(summary, 1):
    print(f"{i:<6} {s['strategy']:<35} ${s['net_earnings']:>14,.2f}")

# Best baseline
baselines = ['immediate_sale', 'equal_batch', 'price_threshold', 'moving_average']
best_baseline = max([s for s in summary if s['strategy'] in baselines],
                   key=lambda x: x['net_earnings'])

print(f"\nBest Baseline: {best_baseline['strategy']} (${best_baseline['net_earnings']:,.2f})")

# Prediction strategies vs baseline
print("\n" + "="*80)
print("PREDICTION STRATEGIES vs BEST BASELINE")
print("="*80)

predictions = [s for s in summary if s['strategy'] not in baselines]
for s in predictions:
    improvement = s['net_earnings'] - best_baseline['net_earnings']
    pct = 100 * improvement / best_baseline['net_earnings']
    marker = "✓" if improvement > 0 else "✗"
    print(f"{marker} {s['strategy']:<35} ${s['net_earnings']:>12,.2f} ({pct:+6.2f}%)")

# Matched pairs
print("\n" + "="*80)
print("MATCHED PAIR ANALYSIS")
print("="*80)

pairs = [
    ('price_threshold', 'price_threshold_predictive'),
    ('moving_average', 'moving_average_predictive')
]

for base_name, pred_name in pairs:
    base_earnings = all_results[base_name][1]
    pred_earnings = all_results[pred_name][1]
    improvement = pred_earnings - base_earnings
    pct = 100 * improvement / base_earnings

    print(f"\n{base_name.upper()}:")
    print(f"  Baseline:   ${base_earnings:,.2f}")
    print(f"  Predictive: ${pred_earnings:,.2f}")
    print(f"  Improvement: ${improvement:,.2f} ({pct:+.2f}%)")
    if improvement > 0:
        print(f"  ✓ Predictions add value!")
    else:
        print(f"  ✗ Predictions hurt performance")

print("\n" + "="*80)
print("EFFICIENCY GAIN: 1,800 trials vs 520,000 grid combinations")
print("99.6% reduction in evaluations (289x fewer)")
print("="*80)

## Save Results

In [ ]:
import json

# Prepare summary for JSON
json_summary = {
    'timestamp': datetime.now().isoformat(),
    'commodity': COMMODITY,
    'model_version': MODEL_VERSION,
    'optimization': {
        'algorithm': 'Optuna TPE',
        'trials_per_strategy': 200,
        'workers': 8,
        'total_trials': len(all_results) * 200
    },
    'costs': {
        'storage_pct_per_day': COMMODITY_CONFIG['storage_cost_pct_per_day'],
        'transaction_pct': COMMODITY_CONFIG['transaction_cost_pct']
    },
    'strategies': {}
}

for name, (params, best_value) in all_results.items():
    # Remove cost params from display
    display_params = {k: v for k, v in params.items()
                     if k not in ['storage_cost_pct_per_day', 'transaction_cost_pct']}
    
    json_summary['strategies'][name] = {
        'net_earnings': float(best_value),
        'best_params': display_params
    }

# Save JSON
json_path = '/Volumes/commodity/trading_agent/files/diagnostic_16_optuna_summary.json'
with open(json_path, 'w') as f:
    json.dump(json_summary, f, indent=2)

print(f"✓ Summary saved: {json_path}")

# Save full results as pickle
pkl_path = '/Volumes/commodity/trading_agent/files/diagnostic_16_optuna_results.pkl'
with open(pkl_path, 'wb') as f:
    pickle.dump(all_results, f)

print(f"✓ Full results saved: {pkl_path}")

print("\n" + "="*80)
print("DIAGNOSTIC 16 OPTUNA OPTIMIZATION COMPLETE")
print("="*80)
print(f"\nOptimized {len(all_results)} strategies")
print(f"Total trials: {len(all_results) * 200:,}")
print(f"vs Grid search: 520,000 combinations")
print(f"Reduction: 99.6% fewer evaluations")
print(f"\nResults saved to Volume for analysis")